In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# blocks output in Colab 💄
%%capture
!pip install patool

In [ ]:
#!pip install gensim spacy nltk

In [4]:
import os
import re
import spacy
import gensim
import patoolib
import numpy as np
import pandas as pd
np.version.version

'1.23.5'

In [5]:
#!pwd
os.chdir('/content/drive/MyDrive/Project4/')
!pwd

/content/drive/MyDrive/Project4


In [6]:
!wget -P /content/drive/MyDrive/model/  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_sg_100_wiki.zip

--2023-10-27 20:34:09--  https://bakrianoo.ewr1.vultrobjects.com/aravec/full_grams_sg_100_wiki.zip
Resolving bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)... 108.61.0.122, 2001:19f0:0:22::100
Connecting to bakrianoo.ewr1.vultrobjects.com (bakrianoo.ewr1.vultrobjects.com)|108.61.0.122|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 507605088 (484M) [application/zip]
Saving to: ‘/content/drive/MyDrive/model/full_grams_sg_100_wiki.zip’

full_grams_sg_100_w 100%[===================>] 484.09M  35.8MB/s    in 13s     

2023-10-27 20:34:23 (36.7 MB/s) - ‘/content/drive/MyDrive/model/full_grams_sg_100_wiki.zip’ saved [507605088/507605088]



In [7]:
filename = "/content/drive/MyDrive/model/full_grams_sg_100_wiki.zip"
patoolib.extract_archive(filename, outdir="/content/drive/MyDrive/Project4/model/")
print("Dataset Extracted Successfully!")

patool: Extracting /content/drive/MyDrive/model/full_grams_sg_100_wiki.zip ...
patool: running /usr/bin/7z x -o/content/drive/MyDrive/Project4/model/ -- /content/drive/MyDrive/model/full_grams_sg_100_wiki.zip


PatoolError: ignored

# Initialize AraVec Spacy Model  (similarity)

https://github.com/bakrianoo/aravec/blob/master/aravec-with-spacy.ipynb

In [ ]:
# Clean/Normalize Arabic Text
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']

    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)

    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)

    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')

    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])

    #trim
    text = text.strip()

    return text

## Load AraVec

In [8]:
# word_embedding_model = gensim.models.Word2Vec.load('models/wikipedia_sg_300')

# load the AraVec model
model = gensim.models.Word2Vec.load("/content/drive/MyDrive/Project4/model/full_grams_sg_100_wiki.mdl")
print("We've",len(model.wv.index_to_key),"vocabularies")

We've 662109 vocabularies


## Export the Word2Vec format + gzip it.

In [9]:
# make a directory called "spacyModel"
%mkdir spacyModel

mkdir: cannot create directory ‘spacyModel’: File exists


In [10]:
# export the word2vec fomart to the directory
model.wv.save_word2vec_format("spacyModel/aravec.txt")

In [ ]:
# using `gzip` to compress the .txt file
!gzip spacyModel/aravec.txt

## Initialize the spaCy model using AraVec vectors


    This will create a folder called /spacy.aravec.model within your current working directory.
    This step could take several minutes to be completed.


In [ ]:
!python -m spacy init vectors ar spacyModel/aravec.txt.gz spacyModel/output/

2023-09-25 19:28:55.613984: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
ℹ Creating blank nlp object for language 'ar'
662109it [00:30, 21956.67it/s]
✔ Successfully converted 662109 vectors
✔ Saved nlp object with vectors to output directory. You can now use
the path to it in your config as the 'vectors' setting in [initialize].
/content/drive/MyDrive/Project4/spacyModel/output


## Run Your AraVec Spacy Model

In [ ]:
# load AraVec Spacy model
#nlp = spacy.load("./spacy.aravec.model/")
nlp = spacy.load("spacyModel/output/")

In [ ]:
# Define the preprocessing Class
class Preprocessor:
    def __init__(self, tokenizer, **cfg):
        self.tokenizer = tokenizer

    def __call__(self, text):
        preprocessed = clean_str(text)
        return self.tokenizer(preprocessed)

In [ ]:
# Apply the `Preprocessor` Class
nlp.tokenizer = Preprocessor(nlp.tokenizer)

## Test the Model

In [ ]:
# Test your model
nlp("قطة").vector

array([-0.26019293, -0.04202315,  0.33389068,  0.6062564 , -0.41907743,
       -0.6454514 ,  0.28226554, -0.5385608 , -0.3224384 , -0.00156201,
        0.00605866, -0.02590263, -0.0519431 ,  0.01232146,  0.5934288 ,
        0.37876508,  0.11991437, -0.14809343, -0.5254807 ,  0.66708595,
       -0.27200884,  0.2092798 ,  0.3665249 ,  0.02955916, -0.43033183,
       -0.31059387, -0.85821354,  0.5033502 ,  0.3000957 ,  0.15807323,
        0.05812804,  0.07539093, -0.41094926,  0.08693237,  0.27560443,
        0.58751583, -0.2914326 , -0.21929309, -0.10528431,  0.2463174 ,
        0.25901592,  0.1769022 ,  0.30446213, -0.42166165,  0.14932899,
        0.33818042, -0.26585534, -0.20072114, -0.20533341, -0.17010476,
        0.13510424,  0.03240122,  0.24817231, -0.07574294, -0.8486026 ,
       -0.29137623, -0.47904313,  0.35913134,  0.4743028 ,  0.01942271,
        0.03372366,  0.1603214 , -0.31512204, -0.51589394, -0.00899144,
        0.12968703,  0.0947319 ,  0.09693876, -0.87194955,  0.21

In [ ]:
egypt = nlp("مصر")
tunisia = nlp("تونس")
apple = nlp("تفاح")

print("egypt Vs. tunisia = ", egypt.similarity(tunisia))
print("egypt Vs. apple = ", egypt.similarity(apple))

egypt Vs. tunisia =  0.5730427440435777
egypt Vs. apple =  0.09378314784708221
